In [ ]:
# Stocks Symbol List Data Source: https://datahub.io/core/nyse-other-listings#python
# Stock Data Source: FinnHub

In [1]:
# packages
# data
import finnhub
from datapackage import Package

# scientific
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# utils
import time
import requests
import json


In [ ]:
################################################################################################

In [2]:
# Gathering data from https://datahub.io/core/nyse-other-listings#python 
# -- on 01.12.2021 8497 symbols were captured
# stocks = []
# package = Package('https://datahub.io/core/nyse-other-listings/datapackage.json')

# # print list of all resources:
# # print(package.resource_names)

# # print processed tabular data (if exists any)
# for resource in package.resources:
#     if resource.descriptor['datahub']['type'] == 'derived/csv':
#         for entry in resource.read():
#             stocks.append(entry[0])

In [3]:
# EXTRA SYMBOLS
weed_stocks = ["GRWG", "JUSHF", "CRLBF", "HRVSF", "MJ", "CURLF", "CGC"]

tech_stocks = ["SNE", "XRX", "CSCO", "SIRI", "ROKU", "ATVI", "EA", "VZ", "T", "TMUS", "AMAT",
              "TXN", "MU", "FB", "MSFT", "SNAP", "AMZN", "TWTR", "AMD", "QCOM", "AVGO", "AAPL",
              "INTC", "NVDA", "NFLX"]

fin_stocks = ["AXP", "V", "MA", "JPM", "DFS", "MS", "BAC", "GS", "SQ", "PYPL", "GDOT"]

oil_chem_stocks = ["NGG", "PCG", "RDS-B", "OXY", "MRO", "DD", "CVX", "DOW", "XOM", "PSX", "COP", 
              "HAL"]

travel_stocks = ["LUV", "MAR", "ABNB", "CCL", "RCL", "DIS", "AAL", "JBLU", "SAVE", "DAL"]

food_consumer_stocks = ["DRI", "GIS", "NKE", "PG", "SHAK", "GRUB", "KDP", "PEP", "KO", "K", "QSR", "CMG",
              "YUM", "BGS", "ACI", "DAR", "KR", "GE", "GOOS", "HD", "LULU", "DPZ", "SBUX", "MMM",
              "WMT", "TGT", "COST", "JACK", "MCD"]

pharma_stocks = ["CVS", "RAD", "UNH", "LLY", "JNJ", "BMY", "AZN", "MRK", "ABBV", "GILD", "PFE"]

defense_infra_stocks = ["GD", "LMT", "NOC", "BA", "CAT", "WM"]

auto_stocks = ["HMC", "TM", "F", "TSLA", "GM"]

all_stocks = (weed_stocks + tech_stocks + fin_stocks + oil_chem_stocks 
                + travel_stocks + food_consumer_stocks + pharma_stocks 
                + defense_infra_stocks + auto_stocks)

# for symb in all_stocks:
#     if str(symb) not in stocks:
#         stocks.append(str(symb))

In [9]:
# scrubbing data of symbols which contain bad_chars = ["$", ";", ":", "!", "*", "."]
# -- run until there is no more output
# bad_chars = ["$", ";", ":", "!", "*", "."]

# for s in stocks:
#     for char in bad_chars:
#         if char in str(s):
#             print(s)
#             stocks.remove(str(s))

In [115]:
display(len(all_stocks))

116

In [ ]:
################################################################################################

In [132]:
df = {}

In [133]:
# Adding company beta and 10_day_avg_volume values to a dataframe
count = 1

# Stock data
for stock in all_stocks:
    try:
        # request company's beta
        r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=' + str(stock) + '&metric=all&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()["metric"]
        count += 1

        beta = metrics["beta"]
        vol = metrics["10DayAverageTradingVolume"]
        if beta == None or vol == None or (str(stock) in df.keys()):
            continue

        # fill dictionary
        if beta > 1.5 and vol <= 100:
            df[str(stock)] = {"beta": beta, "volume_10_day": vol}

    except:
        display(r.json())
        time.sleep(30)
        pass

{'metric': {}, 'metricType': 'all', 'series': {}, 'symbol': 'MJ'}

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

In [137]:
# dataframe to store stock data
data = pd.DataFrame(df)
data = data.transpose()
display(data)

,beta,volume_10_day
GRWG,3.10514,2.72027
CURLF,1.56339,1.16486
CGC,2.32319,1.33443
XRX,1.89355,1.55710
ROKU,1.93547,4.88695
AMD,2.18922,35.03765
DFS,1.76772,1.95352
MS,1.51871,9.15978
BAC,1.55317,47.77609
SQ,2.58161,10.74519


In [138]:
data['adx'] = ''
data['signal'] = ''
data['trending'] = ''
data['l_0'] = ''
data['l_1'] = ''
data['l_2'] = ''
data['l_3'] = ''
data['l_4'] = ''
data['l_5'] = ''

In [139]:
# adding trends, signals, and price levels
count = 1

# Stock data
for stock in data.index:
    try:
        # Aggregate Indicators
        r = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol=' + str(stock) + '&resolution=W&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        count += 1
        
        signal = metrics["technicalAnalysis"]["signal"]
        adx = metrics["trend"]["adx"]
        trending = metrics["trend"]["trending"]

        # add to dataframe
        data.at[str(stock), 'adx'] = adx
        data.at[str(stock), 'signal'] = signal
        data.at[str(stock), 'trending'] = trending

        
        # support and resistance levels -- l_1 == level_1
        r = requests.get('https://finnhub.io/api/v1/scan/support-resistance?symbol=' + str(stock) + '&resolution=W&token=bvt5fmf48v6rku8bvmn0')
        levels = r.json()["levels"]
        
        # add to dataframe iteratively
        for it, l in enumerate(levels):
            data.at[str(stock), 'l_' + str(it)] = l

    except:
        print(str(count) + '...' + str(stock))
        time.sleep(30)
        pass

In [140]:
data = data.sort_values(by=["adx", "signal", "trending"], axis=0, ascending=False)
display(data.head(15))

,beta,volume_10_day,adx,signal,trending,l_0,l_1,l_2,l_3,l_4,l_5
GRWG,3.10514,2.72027,72.799,buy,True,1.5,3.79,7.8213,22.88,,
TSLA,2.11745,42.20043,60.8842,buy,True,35.4,77.92,273,502.49,,
ROKU,1.93547,4.88695,58.2433,strong buy,True,18.35,59.18,120.35,185.44,239.14,
AMD,2.18922,35.03765,47.918,buy,True,9.04,21.04,41.79,59.27,97.98,
DFS,1.76772,1.95352,46.0775,buy,True,23.25,35.05,54.36,64,76.8,92.98
SQ,2.58161,10.74519,44.3238,buy,True,16.11,44.94,74.55,133.81,208.81,
CGC,2.32319,1.33443,38.9562,strong buy,True,4.8995,13.83,27.25,41.68,59.25,
SHAK,1.75100,0.59754,38.5032,strong buy,True,30.01,47.39,64.02,78.47,105.84,
MS,1.51871,9.15978,33.5249,strong buy,True,27.2,27.2,42.24,47.44,51.94,59.38
CURLF,1.56339,1.16486,32.8696,strong buy,True,2.54,4.51,7.06,9.8,13.0779,


In [141]:
data['o'] = ''
data['c'] = ''

In [142]:
# adding current quote -- open and current
for stock in data.index:
    r = requests.get('https://finnhub.io/api/v1/quote?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
    metrics = r.json()
    
    o = metrics['o']
    c = metrics['c']
    
    data.at[str(stock), 'o'] = o
    data.at[str(stock), 'c'] = c


In [143]:
data = data.sort_values(by=["adx", "signal", "trending"], axis=0, ascending=False)
display(data.head(15))

,beta,volume_10_day,adx,signal,trending,l_0,l_1,l_2,l_3,l_4,l_5,o,c
GRWG,3.10514,2.72027,72.799,buy,True,1.5,3.79,7.8213,22.88,,,50.63,49.8542
TSLA,2.11745,42.20043,60.8842,buy,True,35.4,77.92,273,502.49,,,831,853.472
ROKU,1.93547,4.88695,58.2433,strong buy,True,18.35,59.18,120.35,185.44,239.14,,407.9,412.17
AMD,2.18922,35.03765,47.918,buy,True,9.04,21.04,41.79,59.27,97.98,,97.86,95.5
DFS,1.76772,1.95352,46.0775,buy,True,23.25,35.05,54.36,64,76.8,92.98,95.67,94.71
SQ,2.58161,10.74519,44.3238,buy,True,16.11,44.94,74.55,133.81,208.81,,228.51,225.2
CGC,2.32319,1.33443,38.9562,strong buy,True,4.8995,13.83,27.25,41.68,59.25,,31.5,30.84
SHAK,1.75100,0.59754,38.5032,strong buy,True,30.01,47.39,64.02,78.47,105.84,,93.16,96.76
MS,1.51871,9.15978,33.5249,strong buy,True,27.2,27.2,42.24,47.44,51.94,59.38,76.56,75.91
CURLF,1.56339,1.16486,32.8696,strong buy,True,2.54,4.51,7.06,9.8,13.0779,,15.03,14.04


In [ ]:
# plot for beta
fig, ax = plt.subplots()
ax.bar(list(df.keys()), beta, color='g')

ax.set_xticks(range(len(df)))
ax.set_xticklabels(list(df.keys()))
# [1::2] means start from the second element in the list and get every other element
for tick in ax.xaxis.get_major_ticks()[1::2]:
    tick.set_pad(15)
plt.show()

In [ ]:
# plot for volume  
fig, ax = plt.subplots()
ax.bar(list(df.keys()), vol, color='g')

ax.set_xticks(range(len(df)))
ax.set_xticklabels(list(df.keys()))
# [1::2] means start from the second element in the list and get every other element
for tick in ax.xaxis.get_major_ticks()[1::2]:
    tick.set_pad(15)
plt.show()